In [19]:
# !pip install datetime
from datetime import date 
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time
import json
import pandas as pd
import requests
print("Done")

ModuleNotFoundError: No module named 'selenium'

In [5]:
# SCRIPT FOR GETTING ANY MEAL 

def getMeal(meal, hall, savefile):

    mealId = {"Breakfast": 1, "Lunch": 2, "Dinner": 3}
    hallId = {"Allison": 1, "Sargent": 2, "West": 3, "East": 4, "Elder": 5}
    
    # Enable Performance Logging of Chrome.
    desired_capabilities = DesiredCapabilities.CHROME
    desired_capabilities["goog:loggingPrefs"] = {"performance": "ALL"}

    # Create the webdriver object and pass the arguments
    options = webdriver.ChromeOptions()

    # Chrome will start in Headless mode
    options.add_argument('headless')

    # Ignores any certificate errors if there is any
    options.add_argument("--ignore-certificate-errors")

    # Startup the chrome webdriver with executable path and
    # pass the chrome options and desired capabilities as
    # parameters.
    driver = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver",
                              chrome_options=options,
                              desired_capabilities=desired_capabilities)

    driver.get("https://dineoncampus.com/northwestern/whats-on-the-menu")
    
    # Sleeps for 10 seconds
    time.sleep(10)
    
    driver.find_element_by_xpath("//*[@class='btn dropdown-toggle btn-outline-secondary btn-lg']").click()
    
    time.sleep(10)
    
    driver.find_element_by_xpath(f"/html/body/div[1]/div[2]/main/div/div/div[2]/div[1]/div/div[1]/div/ul/li[1]/ul/li[{hallId[hall]}]/button").click()
    
    time.sleep(10)
    
    driver.find_element_by_xpath(f"//*[@role='presentation' and @class='nav-item']//a[@aria-posinset='{mealId[meal]}']").click()
    
    time.sleep(10)

    # Gets all the logs from performance in Chrome
    logs = driver.get_log("performance")


    # Opens a writable JSON file and writes the logs in it
    with open(savefile, "w", encoding="utf-8") as f:
        f.write("[")


        # Iterates every logs and parses it using JSON
        for log in logs:
            network_log = json.loads(log["message"])["message"]

            # Checks if the current 'method' key has any
            # Network related value.
            if("Network.response" in network_log["method"]
                    or "Network.request" in network_log["method"]
                    or "Network.webSocket" in network_log["method"]):

                # Writes the network log to a JSON file by
                # converting the dictionary to a JSON string
                # using json.dumps().

                f.write(json.dumps(network_log)+",")
        f.write("{}]")

    print("Quitting Selenium WebDriver")
    driver.quit()

    # Read the JSON File and parse it using
    # json.loads() to find the urls containing images.
    json_file_path = savefile
    with open(json_file_path, "r", encoding="utf-8") as f:
        logs = json.loads(f.read())

    # Iterate the logs
    answer = []
    for log in logs:

        # Except block will be accessed if any of the
        # following keys are missing.
        try:
            # URL is present inside the following keys
            url = log["params"]["request"]["url"]

            # Checks if the extension is .png or .jpg
#             if "date=" in url:
#                 return url
            if meal == "Breakfast" and "date=" in url:
                return url
            answer.append(url)
        except Exception as e:
            pass
    
    return answer[-1]
print("Done")

Done


In [7]:
def fetchData(meal, hall):
    savefile = str(meal) + "_" + str(hall) + "_" + str(date.today()) + ".json"
    data = requests.get(
        getMeal(meal, hall, savefile), 
        headers=
        {
            "Content-Type":"text", 
            "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Mobile Safari/537.36",
            "Access-Control-Allow-Origin": "*" 
        }
    )
    return json.loads(data.text)


In [8]:
# EXPLORATION
data = fetchData("Lunch", "Allison")
data.keys()

# ONLY MENU AND PERIOD SEEMS TO MATTER
data['menu'].keys()

# ONLY DATA_MENU_PERIODS MATTERS FROM MENU
data['periods']

# NOT SURE IF DATA PERIODS ACTUALLY MATTERS
data['menu']['periods'].keys()

# ONLY DATA_MENU_PERIODS_CATEGORIES MATTERS FROM MENU
data['menu']['periods']['categories']

<ipython-input-5-3057ab24bbd1>:23: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver",
<ipython-input-5-3057ab24bbd1>:23: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver",
<ipython-input-5-3057ab24bbd1>:32: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath("//*[@class='btn dropdown-toggle btn-outline-secondary btn-lg']").click()
<ipython-input-5-3057ab24bbd1>:36: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(f"/html/body/div[1]/div[2]/main/div/div/div[2]/div[1]/div/div[1]/div/ul/li[1]/ul/li[{hallId[hall]}]/button").click()
<ipython-input-5-3057ab24bbd1>:40: DeprecationWarning: find_element_by_xpath is 

Quitting Selenium WebDriver


[{'id': '6292810cb63f1e1beba9cdeb',
  'name': 'Comfort 1',
  'sort_order': 0,
  'items': [{'id': '61f4791f8f3eb63e484699c9',
    'name': 'Steamed Broccoli',
    'mrn': '103333',
    'rev': '0',
    'mrn_full': '103333.0',
    'desc': 'Freshly steamed seasoned broccoli',
    'webtrition_id': 479824468,
    'sort_order': 2,
    'portion': '1/2 cup',
    'qty': None,
    'ingredients': 'Broccoli Fresh Florets, Salt Bulk, Spice Pepper Black Ground',
    'nutrients': [{'id': '62929a10a9f13a00c5b6a663',
      'name': 'Calories',
      'value': '25',
      'uom': 'kcal',
      'value_numeric': '25'},
     {'id': '62929a10a9f13a00c5b6a664',
      'name': 'Protein (g)',
      'value': '2',
      'uom': 'g',
      'value_numeric': '2'},
     {'id': '62929a10a9f13a00c5b6a665',
      'name': 'Total Carbohydrates (g)',
      'value': '5',
      'uom': 'g',
      'value_numeric': '5'},
     {'id': '62929a10a9f13a00c5b6a666',
      'name': 'Sugar (g)',
      'value': '1',
      'uom': 'g',
      'val

In [10]:
def genDataTable(meal, hall):
    data = fetchData(meal, hall)
    
    df = pd.DataFrame()
    df['name'] = []

    all_items = [a['items'] for a in data['menu']['periods']['categories']]
    flat_items = []

    for i in all_items:
        for j in i:
            flat_items.append(j)

    flat_items

    select_keys = ['name', 'desc', 'portion', 'ingredients', 'nutrients', 'calories']
    for key in select_keys:
        df[key] = [a[key] for a in flat_items]
        
    nutrient_names = [a['name'] for a in df['nutrients'][0]]
    for nutrient in nutrient_names:
        df[nutrient] = [next((x['value'] for x in df['nutrients'][a] if x['name'] == nutrient_names[0]), None) for a in range(len(df['nutrients']))]
    
    df.to_csv(str(meal)+ "_" + str(hall) + "_" + str(date.today()) + ".csv")
    df.to_json(str(meal)+ "_" + str(hall) + "_" + str(date.today()) + ".json")

    print("Done")

genDataTable("Dinner", "Sargent")

<ipython-input-5-3057ab24bbd1>:23: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver",
<ipython-input-5-3057ab24bbd1>:23: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path="/usr/local/bin/chromedriver",
<ipython-input-5-3057ab24bbd1>:32: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath("//*[@class='btn dropdown-toggle btn-outline-secondary btn-lg']").click()
<ipython-input-5-3057ab24bbd1>:36: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(f"/html/body/div[1]/div[2]/main/div/div/div[2]/div[1]/div/div[1]/div/ul/li[1]/ul/li[{hallId[hall]}]/button").click()
<ipython-input-5-3057ab24bbd1>:40: DeprecationWarning: find_element_by_xpath is 

Quitting Selenium WebDriver
Done
